# Making WFI Mosaics with RomanCal

***

## Server Information

**IMPORTANT: To run this tutorial, please make sure you are logged in the RRN with a large server.**

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports
 Libraries used
- *romancal* for creating association files and running the pipeline
- *astropy.visualization* for creating image normalizations
- *roman_datamodels* for opening Roman WFI ASDF files
- *glob* for creating lists of files
- *os* for checking if files exist
- *json* for creating association files
- *s3fs* for streaming files from an S3 bucket

In [ ]:
import roman_datamodels as rdm
from romancal.pipeline import MosaicPipeline
from romancal.associations import asn_from_list
from astropy.visualization import simple_norm
import s3fs
import glob
import os
import json
import matplotlib.pyplot as plt

## Introduction
The purpose of this notebook is to combine multiple Level 2 (L2; calibrated rate image) data products into a Level 3 (L3; distortion-corrected, co-added image) product. Details about the Roman data levels can be found [here](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). The pipeline for combining images is called the Mosaic Pipeline. Please review the [RDox pages](https://roman-docs.stsci.edu/data-handbook-home/roman-stsci-data-pipelines/mosaic-level-pipeline) for details. It consists of the following steps:

1. **Flux step:** Transform instrumental units of data numbers per second (DN/s) into physical surface brightness units of megaJanskys per steradian (MJy/sr).
2. **Skymatch step:** Determine scaling factors necessary to match the flux levels in the overlapping sections of the input images. The images will be scaled by these factors later during the combination process.
3. **Outlier detection step:** Examine the input images to detect and flag outliers in the pixel values for a given sky position.
4. **Resample step:** Use the drizzle algorithm to combine the input data products. Reject outliers and other undesirable pixels based on data quality flags and oversample the pixels if indicated.

**Note:** L3 data products in the Roman Archive will be tessellated on the sky such that individual L3 files ("sky cells") will be small with some overlap between adjacent cells. More information is available on the [RDox pages on sky tessellation](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/skymap-tessellation). In this tutorial, we will not utilize tessellation. The tutorial will be updated in the future to demonstrate the creation of both tessellated and non-tessellated products.

***

## Tutorial Data
In this tutorial, we use L2 WFI data files produced by RomanCal processing of L1 simulated files created with Roman I-Sim. If you have already worked through the tutorials [Simulating WFI Imaging Data with Roman I-Sim](../romanisim/romanisim.ipynb) and [Calibrating WFI Exposures with RomanCal](../exposure_pipeline/exposure_pipeline.ipynb), you may already have these files saved locally. If not, these files are also available in the Nexus S3 bucket. For more information on how to access these data, see the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial.

The L2 products used in this tutorial represent data from one WFI detector (WFI11) across four different pointings, each with small dithers. These dithers are optimized for gap-filling between detectors rather than for oversampling. Therefore, in this tutorial we will generate a mosaicked L3 product using the native pixel scale.

## Making WFI Mosaics

### Generating the Association File

Association table files (often abbreviated "association files" or "ASN files") are JavaScript Object Notation (JSON) formatted files containing a list of the input products to be processed together, and the output products to be created. In the Mosaic Pipeline, association files specifically enumerate the input L2 products that are used to create a single L3 product. For more information, please visit the [RDox pages on association files](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products/associations).

The association generator function `asn_from_list()` can be used to create a properly formatted JavaScript Object Notation (JSON) file for use with the Mosaic Pipeline. The code takes as input a list of L2 products and the name of the L3 output product. You can include either one or many L2 products as input. For use cases that do use a single input file, you can skip several pipeline steps (skymatch and outlier_detection).

If you have not completed the previous [Exposure Pipeline](../exposure_pipeline/exposure_pipeline.ipynb) tutorial, then you can download the data products from the Nexus S3 bucket. The code cell below will check if you have the files saved on disk and, if not, it will retrieve them from the S3 bucket.

In [ ]:
# Copy the files from the S3 bucket if they are not in local storage
input_files = ['r0003201001001001004_0001_wfi11_f106_cal.asdf',
               'r0003201001001001004_0002_wfi11_f106_cal.asdf',
               'r0003201001001001004_0003_wfi11_f106_cal.asdf',
               'r0003201001001001004_0004_wfi11_f106_cal.asdf']

for f in input_files:
    if not os.path.exists(f):
        fs = s3fs.S3FileSystem(anon=True)
        asdf_dir_uri = 's3://stpubdata/roman/nexus/soc_simulations/tutorial_data/'
        asdf_file_uri = asdf_dir_uri + f
        fs.get(asdf_file_uri, f'./{f}')

Next we can create the association JSON file using the `asn_from_list()` function:

In [ ]:
# Collect our input files and make the L3 output filename
input_files = glob.glob('r0003*_????_wfi11_f106_cal.asdf')
product_name = 'my_roman_mosaic'

# Create the ASN JSON object
asn = asn_from_list.asn_from_list([(i, 'science') for i in input_files], 
                                  product_name=product_name, 
                                  with_exptype=True, target='')

Now let's take a look at the JSON object we just created. It should contain a listing of each of our input files as well as the output product name: 

In [ ]:
asn

We can also save the ASN file to disk:

In [ ]:
# Write the ASN file to disk
_, serialized = asn.dump(format='json')
with open('my_roman_mosaic_asn.json', 'w') as f:
    f.write(serialized)

### Running the Mosaic Pipeline

Next we run the Mosaic Pipeline using the `MosaicPipeline` class. As with the L1 to L2 Exposure Pipeline, there are many optional arguments to customize the behavior of the pipeline. At this time, we will not delve into these optional parameters. The simulated input files are based on a simple gap-filling dither pattern rather than sub-pixel dithering optimized for the point spread function(PSF). As a result, we use the default native sampling of the WFI detectors. In the future, additional details may be provided to explore optimizing the Mosaic Pipeline output.

If we had only a single input image, this is where we would skip the skymatch and outlier detection steps by passing the optional argument `steps={'skymatch': {'skip': True}, 'outlier_detection': {'skip': True}}`. For now, we will skip the source catalog step that would normally make the single-band source catalog and segmentation map, which are Level 4 (L4; high-level extracted information) files.

**Note:** The following cell will take several minutes to complete.

In [ ]:
result = MosaicPipeline.call('my_roman_mosaic_asn.json', save_results=True, steps={'source_catalog': {'skip': True}})

Now let's open the mosaic file we've created and take a look at what's inside:

In [ ]:
mos_file = rdm.open('my_roman_mosaic_coadd.asdf')
mos_file.info()

In [ ]:
norm = simple_norm(mos_file.data, 'asinh', vmin=0.01, vmax=3.5)
fig, ax = plt.subplots(figsize=(20, 15))
ax.imshow(mos_file.data, origin='lower', norm=norm)

An L3 file contains several extensions that may be of interest:
- data: The mosiac image in units of MegaJanskys per steradian
- context: An image showing the input images that contributed to each output pixel
- err: The error image
- weight: A weight map showing the relative coverage on the sky from the input images and how they contributed to the final mosaic

For more information on the how arrays are computed by the drizzle algorithm, we suggest consulting the [DrizzlePac Handbook](https://www.stsci.edu/files/live/sites/www/files/home/scientific-community/software/drizzlepac/_documents/drizzlepac-handbook.pdf). While the DrizzlePac Handbook is written for HST, many of the same terms and principles apply to Roman as the underlying algorithm is the same.

## Additional Resources
- [RomanCal Documentation](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)
- [DrizzlePac Handbook](https://www.stsci.edu/files/live/sites/www/files/home/scientific-community/software/drizzlepac/_documents/drizzlepac-handbook.pdf)

***

## About this Notebook
**Author:** Tyler Desjardins\
**Updated On:** 2025-12-09

<table width="100%" style="border:none; border-collapse:collapse;">
  <tr style="border:none;">
    <td style="border:none; width:180px; white-space:nowrap;">
       <a href="#top" style="text-decoration:none; color:#0066cc;">↑ Top of page</a> 
    </td>
    <td style="border:none; text-align:center;">
       <img src="../../roman_logo.png" width="50">
    </td>
    <td style="border:none; text-align:right;">
       <img src="../../stsci_logo2.png" width="90">
    </td>
  </tr>
</table>